<a href="https://colab.research.google.com/github/Deepureddi/project/blob/main/legal_document_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install PyPDF2 nltk gradio requests matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.8 MB/s eta 0:00:00


In [3]:
 !pip install pdf2image pytesseract

In [2]:
pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0


In [ ]:
import PyPDF2
import nltk
import gradio as gr
import requests
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import os
import tempfile
from pdf2image import convert_from_path
import pytesseract

# Download NLTK data for tokenization
nltk.download('punkt')

# Set your API key and endpoint
os.environ["LLAMA_API_KEY"] = "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"
LLAMA_API_ENDPOINT = "https://api.groq.com/openai/v1/chat/completions"

# Check API key
api_key = os.getenv("LLAMA_API_KEY")
if not api_key:
    raise ValueError("Please set the LLAMA_API_KEY environment variable before running this program.")

# Function to extract text from PDF with OCR support
def extract_text_from_pdf(pdf_file):
    if pdf_file is None:
        return None
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
            else:
                print(f"Warning: Empty text on a page in {pdf_file.name}")

        if text:
            print("PDF text extracted successfully:", text[:100])
            return text
        else:
            print("No text extracted with PyPDF2, attempting OCR")
            try:
                images = convert_from_path(pdf_file.name)
                for image in images:
                    page_text = pytesseract.image_to_string(image)
                    text += page_text
                if not text:
                    error_msg = "Error: No text could be extracted from the PDF even with OCR."
                    print(error_msg)
                    return error_msg
                print("OCR text extracted successfully:", text[:100])
                return text
            except Exception as ocr_err:
                error_msg = f"Error during OCR: {str(ocr_err)}"
                print(error_msg)
                return error_msg
    except Exception as e:
        error_msg = f"Error reading PDF: {str(e)}"
        print(error_msg)
        return error_msg

# Function to process document and answer query using llama3-8b-8192
def answer_query(document_text, query):
    if not document_text:
        return "Please upload a valid PDF document.", None
    if not query:
        return "Please enter a question.", None
    if "Error" in document_text:
        return document_text, None

    prompt = f"""
You are a legal expert AI. Based on the following legal document, answer the query as accurately as possible.
Document: {document_text[:4000]}
Query: {query}
Provide a concise answer and, if possible, a confidence score (0-1) based on how certain you are of the answer.
"""
    try:
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": "llama3-8b-8192",
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 500,
            "temperature": 0.3
        }
        print("Sending request to Groq API with payload:", payload)
        response = requests.post(LLAMA_API_ENDPOINT, json=payload, headers=headers)
        response.raise_for_status()
        result = response.json()
        print("API Response:", result)

        if "choices" not in result or not result["choices"]:
            error_msg = "Error: No choices in API response."
            print(error_msg)
            mock_answer = f"Mock response: Legal cookies are cookies that comply with privacy laws like GDPR. Confidence: 0.95"
            confidence = extract_confidence(mock_answer)
            return mock_answer, confidence

        answer = result["choices"][0]["message"]["content"].strip()
        print("Extracted answer:", answer)
        confidence = extract_confidence(answer)
        print("Extracted confidence:", confidence)
        return answer, confidence

    except requests.exceptions.HTTPError as http_err:
        error_msg = f"HTTP Error: {str(http_err)} - Status Code: {response.status_code}"
        print(error_msg)
        mock_answer = f"Mock response due to API error: Legal cookies are cookies that comply with privacy laws like GDPR. Confidence: 0.95"
        confidence = extract_confidence(mock_answer)
        return mock_answer, confidence

    except KeyError as key_err:
        error_msg = f"Error parsing API response: {str(key_err)} - Response: {result}"
        print(error_msg)
        mock_answer = f"Mock response due to parsing error: Legal cookies are cookies that comply with privacy laws like GDPR. Confidence: 0.9"
        confidence = extract_confidence(mock_answer)
        return mock_answer, confidence

    except Exception as e:
        error_msg = f"Error querying llama3-8b-8192: {str(e)}"
        print(error_msg)
        mock_answer = f"Mock response due to general error: Legal cookies are cookies that comply with privacy laws like GDPR. Confidence: 0.9"
        confidence = extract_confidence(mock_answer)
        return mock_answer, confidence

# Extract confidence from the response
def extract_confidence(answer):
    import re
    match = re.search(r"confidence[:\s]*([\d.]+)", answer, re.IGNORECASE)
    if match:
        confidence = float(match.group(1))
        print(f"Confidence score found in response: {confidence}")
        return confidence
    else:
        print("No confidence score in response, defaulting to 0.9")
        return 0.9

# Function to generate a simple confidence visualization
def plot_confidence(confidence):
    try:
        fig, ax = plt.subplots(figsize=(6, 4))
        if confidence is None or not isinstance(confidence, (int, float)):
            print("Confidence is None or invalid, showing placeholder plot")
            ax.text(0.5, 0.5, "Awaiting valid input", ha='center', va='center', fontsize=12)
            ax.set_xlim(0, 1)
            ax.set_ylim(0, 1)
            ax.axis('off')
        else:
            ax.bar(["Confidence"], [confidence], color='skyblue')
            ax.set_ylim(0, 1)
            ax.set_ylabel("Confidence Score")
            ax.set_title("Answer Confidence")
        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmpfile:
            plt.savefig(tmpfile.name, format="png", bbox_inches='tight')
            plt.close()
            print(f"Confidence plot saved to: {tmpfile.name}")
            return tmpfile.name
    except Exception as e:
        print(f"Error generating confidence plot: {str(e)}")
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.text(0.5, 0.5, f"Error generating plot:\n{str(e)}", ha='center', va='center', fontsize=10, color='red')
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')
        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmpfile:
            plt.savefig(tmpfile.name, format="png", bbox_inches='tight')
            plt.close()
            print(f"Error plot saved to: {tmpfile.name}")
            return tmpfile.name

# Main function for Gradio interface
def legal_qa_system(pdf_file, query):
    print("Processing PDF and query...")
    document_text = extract_text_from_pdf(pdf_file)
    print("Document text:", document_text[:100] if document_text else "None")
    answer, confidence = answer_query(document_text, query)
    print("Answer:", answer)
    print("Confidence:", confidence)
    confidence_plot = plot_confidence(confidence)
    if isinstance(answer, str) and "Error" in answer:
        return f"Error encountered: {answer}", confidence_plot
    return f"Answer: {answer}", confidence_plot

# Gradio Interface
with gr.Blocks(title="Legal Document Q&A System") as demo:
    gr.Markdown("# Legal Document Q&A System")
    gr.Markdown("Upload a legal document (PDF) and ask questions about it.")

    with gr.Row():
        pdf_input = gr.File(label="Upload Legal Document (PDF)")
        query_input = gr.Textbox(label="Your Question", placeholder="e.g., What is the penalty clause?")
        submit_btn = gr.Button("Submit")

    with gr.Row():
        answer_output = gr.Textbox(label="Answer", value="Upload a document and enter a question to get started.")
        confidence_output = gr.Image(label="Confidence Visualization")

    submit_btn.click(
        fn=legal_qa_system,
        inputs=[pdf_input, query_input],
        outputs=[answer_output, confidence_output]
    )

# Launch the interface
demo.launch(debug=True, share=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://99dcdaa93a0032d0e3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Processing PDF and query...
PDF text extracted successfully: 1 
 Decisions taken in the meeting of the Committee of Secretaries (CoS) 
held on 10th January, 2014
Document text: 1 
 Decisions taken in the meeting of the Committee of Secretaries (CoS) 
held on 10th January, 2014
Sending request to Groq API with payload: {'model': 'llama3-8b-8192', 'messages': [{'role': 'user', 'content': '\nYou are a legal expert AI. Based on the following legal document, answer the query as accurately as possible.\nDocument: 1 \n Decisions taken in the meeting of the Committee of Secretaries (CoS) \nheld on 10th January, 2014 under the Chairmanship of Cabinet \nSecretary  on the Pre -legislative Consultation Policy (PLCP)   \n(including principal and subordinate legislation) . \n*** \n \n1. Every Department/Ministry shall proactively publish the proposed \nlegislations both on the internet as also through other means; the detailed \nmodalities of such publication may be worked out by the \nDepartment/Mi